In [ ]:
# Imports

import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uproot


In [ ]:
# Add ROOT file path+name
rfile_name = "/Users/alexanderantonakis/Software/SBND-PILEUP-ANALYSIS/Data/pmt_opflash_ntuple_bnblight_test.root"

inFile = uproot.open(rfile_name)
event_tree = inFile["ana/events"]
print("Available Keys/Branches:")
print(event_tree.keys())

In [ ]:
B = np.arange(-10, 10, 0.1)

# H is going to be the histogram
# E is going to be the edges of the histogram
H = 0
E = 0

# Set Chunk Size
chunk_size = 1000
# Loop over the events in chunks


for i, df_chunk in enumerate(event_tree.iterate(
        ["run", "subrun", "event", "flash_id", "flash_time"],
        step_size=chunk_size,
        library="pd"
)):
    print(f"Processing chunk {i+1}")
    # Process the chunk

    f_time_v = df_chunk["flash_time"].values
    f_time_flat = np.concatenate(f_time_v)
    f_time_flat = f_time_flat[(f_time_flat > -10) & (f_time_flat < 10)]

    h, edges = np.histogram(f_time_flat, bins=B)
    h = np.array(h, dtype=float)
    if i == 0:
        H = h
        E = edges
    else:
        H += h

print("done")

In [ ]:
bin_centers = (E[:-1] + E[1:]) / 2
bin_width = E[1] - E[0]

plt.errorbar(bin_centers, H, xerr=np.zeros_like(bin_centers)+(bin_width/2), yerr=np.sqrt(H), fmt='o', c='b')

plt.xlabel("Flash Time (us)", fontsize=14)
plt.ylabel("Counts", fontsize=14)
plt.yscale("log")
plt.show()